In [1]:
import os
import numpy as np
import pandas as pd
import slingpy as sp
from genedisco.datasets.features.hgnc_names import HGNCNames
from slingpy.data_access.data_sources.hdf5_tools import HDF5Tools
from slingpy.data_access.data_sources.hdf5_data_source import HDF5DataSource
from slingpy.data_access.data_sources.abstract_data_source import AbstractDataSource

import matplotlib.pyplot as plt
from screen import ScreenData
import scanpy as sc
%matplotlib inline
import matplotlib

/dfs/user/yhr/causal/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
file_path = '/dfs/user/yhr/genedisco/genedisco/datasets/screens/sanchezetal2021_21days.csv'

In [41]:
df = pd.read_csv(file_path, sep=",", index_col="Symbol")

gene_names, data = df.index.values.tolist(), df[['RSA_Down']].values.astype(np.float32)

name_converter = HGNCNames(save_directory)
gene_names = name_converter.update_outdated_gene_names(gene_names)
data_df = pd.DataFrame(data)
            
data_df.index = gene_names
data_df = data_df.groupby(data_df.index).mean()
gene_names, data = data_df.index.values.tolist(), data_df.values.astype(np.float32)

In [50]:
sanchez_df = df.loc[:,['RSA_Down']].reset_index().rename(columns={'Symbol':'Gene', 
                                                     'RSA_Down':'Score'})

In [51]:
sanchez_df.to_csv('/dfs/user/yhr/AI_RA/research_assistant/datasets/ground_truth_Sanchez2021.csv')

### Genedisco/Slingpy dataloader

In [19]:
screen_name = 'crispra_GD2_D22'
h5_file = os.path.join(save_directory, f"steinhart_2024_{screen_name}.h5")
if not os.path.exists(h5_file):
    df = pd.read_csv(f'/dfs/user/yhr/AI_RA/research_assistant/datasets/ground_truth_Steinhart_{screen_name}.csv')
    df = df.rename(columns={'0':'Gene_name', '1':'Score'})

    df = df.set_index('Gene_name')
    
    gene_names = df.index.values.tolist()
    name_converter = HGNCNames(save_directory)
    gene_names = name_converter.update_outdated_gene_names(gene_names)
    df.index = gene_names

    # Merge duplicate indices by averaging
    df = df.groupby(df.index).mean()
    gene_names, data = df.index.values.tolist(), df[['Score']].values.astype(np.float32)


    HDF5Tools.save_h5_file(h5_file,
                           data,
                           f"steinhart_2024_{screen_name}.h5",
                           column_names=["Score"],
                           row_names=gene_names)
data_source = HDF5DataSource(h5_file, duplicate_merge_strategy=sp.MeanMergeStrategy())

/dfs/user/yhr/causal/lib/python3.8/site-packages/genedisco/datasets/features/hgnc_names.py:95: DtypeWarning: Columns (31,38) have mixed types.Specify dtype option on import or set low_memory=False.
  previous_mapping = self.get_hgnc_mapping(to_id="symbol", from_id=other_id_name)


In [20]:
data_source = HDF5DataSource(h5_file, duplicate_merge_strategy=sp.MeanMergeStrategy())

### AI RA dataloader

In [7]:
for screen_name in ['Adenosine', 'TGFb', 'Cyclosporine', 'Tacrolimus']:
    out_file = os.path.join("/dfs/user/yhr/research_assistant/datasets/ground_truth_Carnevale_2022_{}.csv".format(screen_name))
    df = pd.read_excel('/dfs/project/perturb-gnn/AI_RA/Carnevale22/41586_2022_5126_MOESM5_ESM.xlsx',
                   sheet_name=screen_name)
    df = df.set_index('id')

    df.index = df.index.astype('str')
    rows_to_drop = [v for v in df.index if 'CTRL' in v]
    rows_to_drop = [v for v in df.index if v[:4]=='2021']
    df = df.drop(rows_to_drop)

    gene_names = df.index.values.tolist()
    name_converter = HGNCNames('/dfs/user/yhr/genedisco/genedisco/datasets/screens/')
    gene_names = name_converter.update_outdated_gene_names(gene_names)
    df.index = gene_names

    # Merge duplicate indices by averaging
    df = df.groupby(df.index).mean()
    df['pos|lfc'] = df['pos|lfc'].values.astype(np.float32)
    df = df.loc[:,['pos|lfc']]
    df = df.rename(columns = {'pos|lfc':'log-fold-change'})
    df = df.reset_index().rename(columns={'index':'Gene'})

    df.to_csv(out_file, index=0)

/dfs/user/yhr/causal/lib/python3.8/site-packages/genedisco/datasets/features/hgnc_names.py:95: DtypeWarning: Columns (31,38) have mixed types.Specify dtype option on import or set low_memory=False.
  previous_mapping = self.get_hgnc_mapping(to_id="symbol", from_id=other_id_name)
